In [ ]:
#1.1
import pandas as pd

!gdown --fuzzy "https://drive.google.com/file/d/1eyJ3ZHx-pH-bBN0UJV6L2dj2-V6A-q2-/view?usp=sharing"

!unzip -o Loan_Dataset.zip -d loan_data

train_data = pd.read_csv('loan_data/loan_train.csv')
test_data = pd.read_csv('loan_data/loan_test.csv')
train_data['Dependents'] = train_data['Dependents'].replace(['3+'], '3')
test_data['Dependents'] = test_data['Dependents'].replace(['3+'], '3')
train_data['Dependents'] = pd.to_numeric(train_data['Dependents'], errors='coerce')
test_data['Dependents'] = pd.to_numeric(test_data['Dependents'], errors='coerce')

print("Before processing - Train Data Info:")
train_data_info = train_data.info()

print("\nBefore processing - Test Data Info:")
test_data_info = test_data.info()

train_data = train_data.dropna()
test_data = test_data.dropna()

print("\nAfter dropna - Train Data Info:")
train_data_info_after = train_data.info()

print("\nAfter dropna - Test Data Info:")
test_data_info_after = test_data.info()

from sklearn.preprocessing import LabelEncoder

categorical_columns = ['Gender', 'Married', 'Education', 'Self_Employed', 'Area']

label_encoders ={}

for column in categorical_columns:
    le = LabelEncoder()
    train_data[column] = le.fit_transform(train_data[column])
    test_data[column] = le.transform(test_data[column])
    label_encoders[column] = le
le_status = LabelEncoder()
train_data['Status'] = le_status.fit_transform(train_data['Status'])
label_encoders['Status'] = le_status


print("\nAfter encoding categorical columns - Train Data Info:")
train_data_info_final = train_data.info()

print("\nAfter encoding categorical columns - Test Data Info:")
test_data_info_final = test_data.info()

train_data_info, test_data_info, train_data_info_after, test_data_info_after, train_data_info_final, test_data_info_final



Downloading...
From: https://drive.google.com/uc?id=1eyJ3ZHx-pH-bBN0UJV6L2dj2-V6A-q2-
To: /content/Loan_Dataset.zip
100% 11.2k/11.2k [00:00<00:00, 36.9MB/s]
Archive:  Loan_Dataset.zip
  inflating: loan_data/loan_test.csv  
  inflating: loan_data/loan_train.csv  
Before processing - Train Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Gender              601 non-null    object 
 1   Married             611 non-null    object 
 2   Dependents          599 non-null    float64
 3   Education           614 non-null    object 
 4   Self_Employed       582 non-null    object 
 5   Applicant_Income    614 non-null    int64  
 6   Coapplicant_Income  614 non-null    float64
 7   Loan_Amount         614 non-null    int64  
 8   Term                600 non-null    float64
 9   Credit_History      564 non-null    float64
 10  Ar

(None, None, None, None, None, None)

In [ ]:
#1.2
!pip install pyswarms
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pyswarms as ps

X = train_data.drop('Status', axis=1)
y = train_data['Status']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

def evaluate_features_pso(particles):
    scores = []

    for particle in particles:
        use_features = particle > 0.5

        if sum(use_features) == 0:
            scores.append(1.0)
            continue

        X_train_sel = X_train.iloc[:, use_features]
        X_test_sel = X_test.iloc[:, use_features]

        model = RandomForestClassifier(n_estimators=50, random_state=42, max_depth=15)
        model.fit(X_train_sel, y_train)

        predictions = model.predict(X_test_sel)
        accuracy = accuracy_score(y_test, predictions)

        num_features = sum(use_features)
        fitness = 0.5 * (1 - accuracy) + 0.5 * (num_features / len(particle))

        scores.append(fitness)

    return np.array(scores)

options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}

optimizer = ps.single.GlobalBestPSO(
    n_particles=50,
    dimensions=X.shape[1],
    options=options,
    bounds=(np.zeros(X.shape[1]), np.ones(X.shape[1]))
)

best_fitness_pso, best_solution_pso = optimizer.optimize(evaluate_features_pso, iters=100)


selected_pso = best_solution_pso > 0.5
selected_names_pso = X.columns[selected_pso].tolist()

X_train_final_pso = X_train.iloc[:, selected_pso]
X_test_final_pso = X_test.iloc[:, selected_pso]

final_model_pso = RandomForestClassifier(n_estimators=100, random_state=42)
final_model_pso.fit(X_train_final_pso, y_train)
final_pred_pso = final_model_pso.predict(X_test_final_pso)
final_acc_pso = accuracy_score(y_test, final_pred_pso)

print(f"\nFinal Accuracy: {final_acc_pso*100:.2f}%")
print(f"Best Fitness: {best_fitness_pso:.4f}")

2026-02-10 16:11:07,312 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|100/100, best_cost=0.135
2026-02-10 16:19:04,380 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.1354545454545455, best pos: [0.47210337 0.26692033 0.37594641 0.44921523 0.48943918 0.32958669
 0.41450458 0.45834107 0.11067553 0.66032052 0.25414347]



Final Accuracy: 82.00%
Best Fitness: 0.1355


In [ ]:
#1.3
!pip install deap
import random
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from deap import base, creator, tools, algorithms

X = train_data.drop('Status', axis=1)
y = train_data['Status']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

def evaluate(individual):
    selected_features = [index for index, value in enumerate(individual) if value == 1]

    if len(selected_features) == 0:
        return (1.0,)

    X_train_selected = X_train.iloc[:, selected_features]
    X_test_selected = X_test.iloc[:, selected_features]

    model = RandomForestClassifier(n_estimators=50, random_state=42, max_depth=15)
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_test_selected)

    accuracy = accuracy_score(y_test, y_pred)

    num_features = len(selected_features)
    fitness = 0.5 * (1 - accuracy) + 0.5 * (num_features / len(individual))

    return (fitness,)

if hasattr(creator, "FitnessMin"):
    del creator.FitnessMin
if hasattr(creator, "Individual"):
    del creator.Individual

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=X_train.shape[1])
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

population = toolbox.population(n=50)

hof = tools.HallOfFame(1)

pop, logbook = algorithms.eaSimple(
    population,
    toolbox,
    cxpb=0.9,
    mutpb=0.1,
    ngen=50,
    halloffame=hof,
    verbose=True
)

best_individual = hof[0]
selected_features_ga = [index for index, value in enumerate(best_individual) if value == 1]
selected_names_ga = X_train.columns[selected_features_ga].tolist()

X_train_final_ga = X_train.iloc[:, selected_features_ga]
X_test_final_ga = X_test.iloc[:, selected_features_ga]

final_model_ga = RandomForestClassifier(n_estimators=100, random_state=42)
final_model_ga.fit(X_train_final_ga, y_train)
final_pred_ga = final_model_ga.predict(X_test_final_ga)
final_acc_ga = accuracy_score(y_test, final_pred_ga)

print(f"\nFinal Accuracy: {final_acc_ga*100:.2f}%")
print(f"Best Fitness: {best_individual.fitness.values[0]:.4f}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.0/136.0 kB 2.5 MB/s eta 0:00:00
gen	nevals
0  	50    
1  	48    
2  	40    
3  	46    
4  	46    
5  	43    
6  	42    
7  	46    
8  	46    
9  	44    
10 	47    
11 	46    
12 	50    
13 	50    
14 	42    
15 	47    
16 	50    
17 	48    
18 	48    
19 	48    
20 	40    
21 	48    
22 	50    
23 	48    
24 	44    
25 	40    
26 	50    
27 	43    
28 	47    
29 	46    
30 	47    
31 	39    
32 	48    
33 	48    
34 	43    
35 	48    
36 	46    
37 	43    
38 	46    
39 	47    
40 	44    
41 	48    
42 	47    
43 	41    
44 	44    
45 	48    
46 	44    
47 	46    
48 	46    
49 	48    
50 	48    

Final Accuracy: 82.00%
Best Fitness: 0.1355


In [ ]:
#1.4
from sklearn.model_selection import train_test_split, cross_val_score

cv_scores_pso = cross_val_score(final_model_pso, X_train_final_pso, y_train, cv=5, scoring='accuracy')
mean_accuracy_pso = np.mean(cv_scores_pso)
std_accuracy_pso = np.std(cv_scores_pso)

print("\nPSO Results:")
print(f"Selected {len(selected_names_pso)} features out of {X.shape[1]}: {', '.join(selected_names_pso)}")
print(f"Mean Accuracy: {mean_accuracy_pso * 100:.2f}%")
print(f"Accuracy Std Dev: {std_accuracy_pso * 100:.2f}%")
print(f"Final Accuracy: {final_acc_pso * 100:.2f}%")
print(f"Best Fitness: {best_fitness_pso:.4f}")

cv_scores_ga = cross_val_score(final_model_ga, X_train_final_ga, y_train, cv=5, scoring='accuracy')
mean_accuracy_ga = np.mean(cv_scores_ga)
std_accuracy_ga = np.std(cv_scores_ga)

print("\nGA Results:")
print(f"Selected {len(selected_names_ga)} features out of {X.shape[1]}: {', '.join(selected_names_ga)}")
print(f"Mean Accuracy: {mean_accuracy_ga * 100:.2f}%")
print(f"Accuracy Std Dev: {std_accuracy_ga * 100:.2f}%")
print(f"Final Accuracy: {final_acc_ga * 100:.2f}%")
print(f"Best Fitness: {best_individual.fitness.values[0]:.4f}")


PSO Results:
Selected 1 features out of 11: Credit_History
Mean Accuracy: 79.67%
Accuracy Std Dev: 5.41%
Final Accuracy: 82.00%
Best Fitness: 0.1355

GA Results:
Selected 1 features out of 11: Credit_History
Mean Accuracy: 79.67%
Accuracy Std Dev: 5.41%
Final Accuracy: 82.00%
Best Fitness: 0.1355


In [ ]:
#1.5
print(f"Selected {len(selected_names_pso)} features out of {X.shape[1]}: {', '.join(selected_names_pso)}")
print(f"Selected {len(selected_names_ga)} features out of {X.shape[1]}: {', '.join(selected_names_ga)}")

Selected 1 features out of 11: Credit_History
Selected 1 features out of 11: Credit_History


In [ ]:
#1.6
import random
import matplotlib.pyplot as plt

def evaluate_features_pso(particles):
    scores = []
    for particle in particles:
        use_features = particle > 0.5

        if use_features.sum() == 0:
           scores.append(1.0)
           continue

        X_train_sel = X_train.iloc[:, use_features]
        X_test_sel = X_test.iloc[:, use_features]

        model = RandomForestClassifier(n_estimators=10, random_state=42, max_depth=5)
        model.fit(X_train_sel, y_train)
        pred = model.predict(X_test_sel)
        acc = accuracy_score(y_test, pred)

        num_feat = use_features.sum()
        fitness = 0.5 * (1 - acc) + 0.5 * (num_feat / len(particle))
        scores.append(fitness)

    return np.array(scores)

options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}

optimizer = ps.single.GlobalBestPSO(
    n_particles=50,
    dimensions=X.shape[1],
    options=options,
    bounds=(np.zeros(X.shape[1]), np.ones(X.shape[1]))
)

best_fitness_pso, best_solution_pso = optimizer.optimize(evaluate_features_pso, iters=100, verbose=False)

pso_fitness_history = optimizer.cost_history
pso_iters = np.arange(1, len(pso_fitness_history) + 1)

selected_pso = best_solution_pso > 0.5
selected_names_pso = X.columns[selected_pso].tolist()

X_train_final_pso = X_train.iloc[:, selected_pso]
X_test_final_pso = X_test.iloc[:, selected_pso]

final_model_pso = RandomForestClassifier(n_estimators=100, random_state=42)
final_model_pso.fit(X_train_final_pso, y_train)
final_pred_pso = final_model_pso.predict(X_test_final_pso)
final_acc_pso = accuracy_score(y_test, final_pred_pso)


def evaluate_ga(individual):
    selected_features = [i for i, v in enumerate(individual) if v == 1]

    if len(selected_features) == 0:
       return (1.0,)

    X_train_sel = X_train.iloc[:, selected_features]
    X_test_sel = X_test.iloc[:, selected_features]

    model = RandomForestClassifier(n_estimators=10, random_state=42, max_depth=5)
    model.fit(X_train_sel, y_train)
    pred = model.predict(X_test_sel)
    acc = accuracy_score(y_test, pred)

    num_feat = len(selected_features)
    fitness = 0.5 * (1 - acc) + 0.5 * (num_feat / len(individual))
    return (fitness,)

if hasattr(creator, "FitnessMin"):
    del creator.FitnessMin
if hasattr(creator, "Individual"):
    del creator.Individual

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=X.shape[1])
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate_ga)

population = toolbox.population(n=50)
hof = tools.HallOfFame(1)

stats = tools.Statistics(lambda ind: ind.fitness.values[0])
stats.register("min", np.min)
stats.register("avg", np.mean)

pop, logbook = algorithms.eaSimple(
    population,
    toolbox,
    cxpb=0.9,
    mutpb=0.1,
    ngen=50,
    halloffame=hof,
    stats=stats,
    verbose=False
)

ga_gens = logbook.select("gen")
ga_min_fitness = logbook.select("min")


best_individual = hof[0]
selected_features_ga = [i for i, v in enumerate(best_individual) if v == 1]
selected_names_ga = X.columns[selected_features_ga].tolist()

X_train_final_ga = X_train.iloc[:, selected_features_ga]
X_test_final_ga = X_test.iloc[:, selected_features_ga]

final_model_ga = RandomForestClassifier(n_estimators=100, random_state=42)
final_model_ga.fit(X_train_final_ga, y_train)
final_pred_ga = final_model_ga.predict(X_test_final_ga)
final_acc_ga = accuracy_score(y_test, final_pred_ga)

best_fitness_ga = best_individual.fitness.values[0]

print("PSO Results:")
print(f"Selected {len(selected_names_pso)} features out of {X.shape[1]}: {selected_names_pso}")
print(f"Final Accuracy: {final_acc_pso*100:.2f}%")
print(f"Best Fitness: {best_fitness_pso:.4f}\n")

print("GA Results:")
print(f"Selected {len(selected_names_ga)} features out of {X.shape[1]}: {selected_names_ga}")
print(f"Final Accuracy: {final_acc_ga*100:.2f}%")
print(f"Best Fitness: {best_fitness_ga:.4f}\n")

plt.figure(figsize=(10,6))
plt.plot(pso_iters, pso_fitness_history, marker='x', linestyle='--', label='PSO (best fitness per iter)')
plt.plot(ga_gens, ga_min_fitness, marker='o', linestyle='-', label='GA (min fitness per gen)')
plt.xlabel("Iterations / Generations")
plt.ylabel("Fitness Value")
plt.title("Comparison of PSO and GA Fitness Over Time")
plt.grid(True)
plt.legend()
plt.show()


PSO Results:
Selected 1 features out of 11: ['Credit_History']
Final Accuracy: 82.00%
Best Fitness: 0.1355

GA Results:
Selected 1 features out of 11: ['Credit_History']
Final Accuracy: 82.00%
Best Fitness: 0.1355

